In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from urllib.parse import urlparse

from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By

import time

from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

In [ ]:
options = Options()
# options.add_argument("-headless")
options.add_argument("--no-sandbox")
options.add_argument("disable-infobars")
options.add_argument("--enable-javascript")
options.add_argument("--disable-cookies")
options.add_argument('--proxy-server=%s' % 'insert your proxy here')
firefox_profile = FirefoxProfile()
firefox_profile.set_preference("javascript.enabled", False)
options.profile = firefox_profile
fp = webdriver.FirefoxProfile()
fp.set_preference("network.cookie.cookieBehavior", 0)
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options,firefox_profile=fp)

<ipython-input-434-17f7b75607f0>:8: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  firefox_profile = FirefoxProfile()
<ipython-input-434-17f7b75607f0>:10: DeprecationWarning: Setting a profile has been deprecated. Please use the set_preference and install_addons methods
  options.profile = firefox_profile
<ipython-input-434-17f7b75607f0>:11: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  fp = webdriver.FirefoxProfile()
[WDM] - Downloading:   0%|                                                                 | 0.00/2.58k [00:00<?, ?B/s]
[WDM] - Downloading: 19.0kB [00:00, 41.1kB/s]                                                                          
[WDM] - Downloading:   0%|                                                                 | 0.00/1.58M [00:00<?, ?B/s]<ipython-input-434-17f7b75607f0>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver =

In [ ]:
country='DEU'
url = f"https://ish.messefrankfurt.com/frankfurt/en/exhibitor-search.html?country={country}"
driver.get(url)

In [ ]:
products = ['Bathroom equipment','Bathroom fittings','Kitchen, work room equipment','Water Treatment','Energy storage, storage tanks','Stations, distributors/manifolds','Refrigeration technology','System, room and building automation']

In [ ]:
output={}

def scrape_data():
    for product in products:
        print(f'processing for {product}')
        url = f"https://ish.messefrankfurt.com/frankfurt/en/exhibitor-search.html?country={country}" # + "&start=" +      str((page - 1) * 10)
        driver.get(url)
        time.sleep(4)
        # click on Filter
        driver.find_element(By.XPATH, "//button[@class='ex-search-form__filter-toggle btn btn-default btn-icon-single btn-icon-single-xxs']").click()
        # select Product groups
        time.sleep(2)
        driver.find_element(By.LINK_TEXT, "Product groups").click()
        time.sleep(4)

        driver.find_element(By.LINK_TEXT, f"{product}").click()
        time.sleep(3)
        # select sub category or select all
        driver.find_element(By.LINK_TEXT,'Select all').click()
        time.sleep(4)
        # show all results
        driver.find_element(By.XPATH,'/html/body/main/div[6]/div/div[2]/div/div/div/div/div[1]/div/div/div/button[3]').click()
        time.sleep(4)
        temp = driver.find_elements(By.XPATH,'/html/body/main/div[6]/div/div[2]/div/div/div/div/div[2]/div/div[5]/div[2]/div/div/div/div')
        if len(temp)>1:
            print('something is not right')
        output[product]=temp[0].get_attribute('innerHTML')
        print('processing completed')
#     return output
scrape_data()

processing for Bathroom equipment
processing completed
processing for Bathroom fittings
processing completed
processing for Kitchen, work room equipment
processing completed
processing for Water Treatment
processing completed
processing for Energy storage, storage tanks
processing completed
processing for Stations, distributors/manifolds
processing completed
processing for Refrigeration technology
processing completed
processing for System, room and building automation
processing completed


In [ ]:
company_category_dict={}
def process_output():
    
    for category in products:
        soup=BeautifulSoup(output[category])
        a_list = soup.find_all('a', class_='a-link--no-focus')
        company_sections = soup.find_all('h4', class_='ex-exhibitor-search-result-item__headline')
        company_name_list=[]
        url_list=[]
        for a_tag, company_name_tag in zip(a_list, company_sections):
            url_=a_tag.get('href')
            company_name_=company_name_tag.text
            
            if len(url_)==0:
                print(f'no url found for company {company_name}')
            if len(company_name_)==0:
                print(f'no name found for company {company_name}')
            
            url_list.append(url_)
            company_name_list.append(company_name_)
            
        company_category_dict[category]={'company_names':company_name_list, 'urls': url_list, }    
process_output()
# print(company_category_dict)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 1.58M/1.58M [18:09<00:00, 1.52kB/s]
[WDM] - Downloading: 19.0kB [18:20, 17.7B/s]                                                                           


In [ ]:
import requests

def find_element(path,typed='innerHTML'):
    try:
        output = driver.find_element(By.XPATH,path).get_attribute(typed)
    except:
        output='not found'
    return output

def check_social_exist(keyword,url):
    if url==f'https://www.{keyword}.com/ish_frankfurt':
        return ''
    else:
        return url

def scrape_each_company(url):
    url='https://ish.messefrankfurt.com/'+url
    time.sleep(1)
    # open webpage
    driver.get(url)
    time.sleep(6)
    company_name = find_element('//p[@class="ex-contact-box__address-field-name"]')
    
    street_name = find_element('//p[@class="ex-contact-box__address-field-street"]')
    zip_code = find_element('//span[@class="ex-contact-box__address-field-zip"]')
    city = find_element('//span[@class="ex-contact-box__address-field-city"]')
    telephone = find_element('//a[@class="a-link ex-contact-box__address-field-tel-number"]')
    email = find_element('//a[@class="ex-contact-box__contact-btn btn btn-primary"]','href')
    
    website = find_element('//a[@class="icon icon-news-before ex-contact-box__website-link"]','href')
    
    facebook= find_element('//a[@class="icon-facebook"]','href')
    keyword = 'facebook'
    facebook = check_social_exist(keyword, facebook)
    
    youtube= find_element('//a[@class="icon-youtube"]','href')
    keyword = 'youtube'
    youtube = check_social_exist(keyword, youtube)
    
    instagram = find_element('//a[@class="icon-instagram"]','href')
    keyword = 'instagram'
    instagram = check_social_exist(keyword, instagram)
    
    xing=''
    try:
        xing = find_element('//a[@class="icon-xing"]','href')
        keyword = 'xing'
        xing = check_social_exist(keyword, xing)
    except:
        pass
    
    
    linkedin = find_element('//a[@class="icon-linkedin"]','href')
    keyword = 'linkedin'
    linkedin = check_social_exist(keyword, linkedin)
    return {'company_name':company_name,'street':street_name,'zip_code':zip_code,'city':city,'telephone':telephone,
            'email':email, 'website':website, 'instagram':instagram,'youtube':youtube, 'facebook':facebook, 
            'linkedin':linkedin, 'xing':xing}


In [ ]:
companies_info_dict={}

In [ ]:
# scraping on each url

def url_scraping(dictionary) -> list:
    companies_category_info=[]
    for company_name, url in zip(dictionary['company_names'],dictionary['urls']):
        print(f'processing for company {company_name}')
        
        # if the company is already processed, then find it in companies_info_dict and add it to the current category list
        if len(companies_info_dict)>0 and companies_info_dict.__contains__(company_name):
            companies_category_info.append(companies_info_dict[company_name])
        else:
            company_info = scrape_each_company(url)
            
            companies_category_info.append(company_info)
            companies_info_dict[company_info['company_name']] = company_info
    return companies_category_info

In [ ]:
df_=pd.DataFrame()
df_.to_excel('scrapping_results2.xlsx')

In [ ]:

with pd.ExcelWriter('scrapping_results2.xlsx') as writer:
    
    for product in products:
        print(f'********category: {product}*******')
        time.sleep(5)
        product_name=product
        if product == 'Stations, distributors/manifolds':
            product_name='stations and distributors'
        if product == 'System, room and building automation':
            product_name='System room automation'
        cat = url_scraping(company_category_dict[product])
        df_ = pd.DataFrame(cat)
        df_.to_excel(writer,sheet_name=product_name)
    

********category: Bathroom equipment*******
processing for company a.h.s. GmbH
processing for company Bemis GmbH
processing for company Benefit GmbH
processing for company Bette GmbH & Co. KG
processing for company Blanke Systems GmbH & Co. KG
processing for company BörSting GmbH
processing for company Herbert Burda GmbH
processing for company burgbad GmbH
processing for company Duravit AG
processing for company Electrostar GmbH
processing for company ESS - Easy Sanitary Solutions GmbH
processing for company FGI Germany GmbH & Co. KG
processing for company Fluidmaster (Wisa, Schwab, Liv)
processing for company Geberit Vertriebs GmbH
processing for company Hamberger Sanitary GmbH
processing for company Hennecke GmbH
processing for company Hoesch Design GmbH
processing for company HSK Duschkabinenbau KG
processing for company Jokey Sohland GmbH
processing for company Franz Kaldewei GmbH & Co. KG
processing for company Kermi GmbH Duschdesign
processing for company KWC Aquarotter GmbH - A 